In [1]:
using Random
using LinearAlgebra
using Revise
using RPCholesky
using Plots
using Distributions
using BenchmarkTools

┌ Warning: backend `GR` is not installed.
└ @ Plots ~/.julia/packages/Plots/HyyIK/src/backends.jl:43


In [2]:
h = 0.8;
K(x, y) = exp(-(x - y)^2 / (2 * h^2))

N = 10;
x_pts = LinRange(0, 5, N);

In [3]:
A = [K(x_, y_) for x_ in x_pts, y_ in x_pts];

In [6]:
Random.seed!(100)
F, S = rpcholesky(A, 2);

In [7]:
F

10×2 Matrix{Float64}:
 0.785742     -0.200476
 1.0           0.0
 0.785742      0.525946
 0.381171      0.924504
 0.114162      0.802838
 0.0211097     0.403595
 0.00240992    0.122491
 0.000169857   0.0227635
 7.39135e-6    0.00260366
 1.98575e-7    0.000183645

In [8]:
function rpcholesky2(A, k; τ=1e-8, verbose=false)
    N, _ = size(A)
    F = zeros(N, k)
    S = Int[]
    d = diag(A)
    g = zeros(N)
    s = 0
    trA = tr(A)
    for i in 1:k

        if (sum(d) > 0)
            s = rand(Categorical(d / sum(d)))
        else
            @warn "[$(i)]: NOT A PROBABILITY VECTOR"
            F = F[:, 1:i-1]
            break
        end

        if (i == 1)
            @. g = A[:, s]
        else
            g .= A[:, s] - (F[:, 1:i-1] * F[s, 1:i-1])
        end

        if (g[s] > 0)
            F[:, i] .= g / sqrt(g[s])
            push!(S, s)
        else
            @warn "[$(i)]: NON-POSITIVE PIVOT VALUE"
            F = F[:, 1:i-1]
            break
        end
        @. d -= abs(F[:, i])^2
        @. d = max(d, 0)
        @show d;

        if (sum(d) < τ * trA)
            if (verbose)
                @warn "[$(i)]: TOLERANCE SATISFIED"
            end
            F = F[:, 1:i]
            break
        end

    end

    return F, S

end

rpcholesky2 (generic function with 1 method)

In [9]:
Random.seed!(100)
F, S = rpcholesky2(A, 2);

d = [0.3826092112340901, 0.0, 0.3826092112340901, 0.8547083744544405, 0.9869670925514906, 0.999554382404408, 0.9999941923071192, 0.9999999711487094, 0.999999999945368, 0.9999999999999606]
d = [0.34241869886833987, 0.0, 0.10598960418627995, 0.0, 0.34241869886834, 0.8366657743016316, 0.984990226339434, 0.9994817963745158, 0.9999932208827721, 0.9999999662743252]


In [10]:
S

2-element Vector{Int64}:
 2
 4